## Meteo to nc

In [1]:
from Poseidon.utils import *
from Poseidon.meteo import *
from netCDF4 import Dataset
import datetime

### setup

In [2]:
#define in a dictionary the properties e.g
dic={'lon0':0., # lat/lon window
     'lon1':10.,
     'lat0':28.,
     'lat1':38.}

### extract meteo forcing

In [3]:
#set meteo forcing
m = ecmwf() # initialize

In [4]:
# specify the forecast times ft1,ft2
m.ft1 = 0
m.ft2 = 2 #that is 0,1 hours

In [5]:
# parse the ecmwf grib file.
m.parse(path='/Users/brey/Downloads/20130301.00.tropical_cyclone.grib',**dic)

100%|██████████| 6/6 [00:02<00:00,  2.15it/s]


meteo done


## output 

Below the output specifically for Schism but one can adapt appropriately

In [6]:
#from DataFile class create a file
d = DataFile('/Users/brey/Downloads/test.nc', write=True, create=True,format='NETCDF3_64BIT')

In [7]:
## add OPTIONAL attributes to the nc file
info = {'description' : '',
    'history' :'JRC Ispra European Commission',
    'source' : 'netCDF4 python module'}

In [8]:
d.info(**info)

In [9]:
tstamp = datetime.datetime(2013,3,1,0)

In [10]:
tu = datetime.datetime.strftime(tstamp,'%Y-%m-%d' )
bd = datetime.datetime.strftime(tstamp,'[%Y  %m  %d  %H]' )

In [11]:
time = np.arange(m.ft1,m.ft2)

In [12]:
spfh = np.empty(m.v.shape)

In [13]:
#Write variables
d.write('time',time, **{'long_name': 'Time', 'standard_name': 'time', 'units': tu,'base_date': bd})


d.write('lat',m.lats, **{'long_name': 'Latitude', 'standard_name': 'latitude', 'units': 'degrees_north'})
d.write('lon',m.lons, **{'long_name': 'Longitude', 'standard_name': 'longitude', 'units': 'degrees_east'})
d.write('prmsl',m.p, **{'long_name': 'Pressure reduced to MSL', 'standard_name': 'air_pressure_at_sea_level', 'units': 'Pa'})
d.write('uwind',m.u, **{'long_name': 'Surface Eastward Air Velocity', 'standard_name': 'eastward_wind', 'units': 'm/s'})
d.write('vwind',m.v, **{'long_name': 'Surface Northward Air Velocity', 'standard_name': 'northward_wind', 'units': 'm/s'})
d.write('spfh',spfh, **{'long_name': 'Surface Specific Humidity (2m AGL)', 'standard_name': 'specific_humidity', 'units': '1'})
d.write('stmp',spfh, **{'long_name': 'Surface Temperature', 'standard_name': 'surface temperature', 'units': 'degrees'})

In [14]:
#close
d.close()

In [15]:
#check
q = Dataset('/Users/brey/Downloads/test.nc')

In [16]:
q

<type 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_64BIT_OFFSET data model, file format NETCDF3):
    source: netCDF4 python module
    description: 
    history: JRC Ispra European Commission
    dimensions(sizes): x(2), x2(75), y(75), z(75)
    variables(dimensions): int32 time(x), float64 lat(x2,y), float64 lon(x2,y), float64 prmsl(x,y,z), float64 uwind(x,y,z), float64 vwind(x,y,z), float64 spfh(x,y,z), float64 stmp(x,y,z)
    groups: 